In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')

In [2]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))  
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]
def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
       
        try:
            if i == 0:
                non = not non
            elif i == -1:
                yield int(s) if s.isdigit() else s if case else s.lower()
                s = ''
            else:
                x, s = s[:i], s[i:]
                yield int(x) if x.isdigit() else x if case else x.lower()
        except:
            print(i)

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))

def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return tmp
    else:
        return numbers
     
    
    
# Convert a fraction that is seperated by a space "1 1/3"
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
            return num
        else:
            return numbers
    except:
        return numbers
    


#Convert a fraction that is seperated by a dash "1-1/2"
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers
# tubes_new['height_new'] = tubes_new['height_new'].apply(lambda x: remove_fraction(x))
#########################################################################





# CONVERTING FRACTIONS, WHOLE NUMBERS, AND NUMBERS WITH LETTERS
#########################################################################
#Converts the number to a decimal if it has a decimal number but leaves off .0 if it is an int. Be sure to conver it back to a string and add -"in"
def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()
# from decimal import *
# TWOPLACES = Decimal(10) ** -2
# tubes_new['height_new']=tubes_new['height_new'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))



 
# converts the value to a float if possible and skips the value if not   
def convert_float(val):
    try:
        return float(val)
    except:
        return str(val)
    
    
    

# Definition to round a number
def round_string_float(x):
    try:
        return remove_exponent(Decimal(x).quantize(TWOPLACES))
    except:
        return x   
    
# Excel might convert 8-1/2 to 08/01/2002 and we need to conver this back into 8.5  
def date_to_decimal(numbers):
    if numbers.find('/') > -1:
        try:
            numbers=str(numbers)
            one, two,three = numbers.split('/')
            tmp = float(two)/float(three[-1])
            num = int(one) + tmp
            return str(num)
        except:
            return numbers
    else:
        return numbers

# Convert a fraction "1/4" that is written as a date "1-4-2021"
def remove_basic_date(numbers):
    if numbers.find('-2021') > -1:
        first, second, year = numbers.split('-')
        num=float(first)/float(second)
        return str(num)
    else:
        return numbers
    

# Convert a fraction "1/4" that is written as a date "2021-01-04"
def remove_date_year_first(numbers):
    if numbers.find('2021-') > -1:
        year, first, second = numbers.split('-')
        num=float(first)/float(second)
        return str(num)
    else:
        return numbers
# Convert a fraction "1/4" that is written as a date "1/4/2021"    
def remove_date_year_last(numbers):
    if numbers.find('/2021') > -1:
        first, second,year = numbers.split('/')
        num=float(first)/float(second)
        return str(num)
    else:
        return numbers
    
regex_year_month_day = r"(?i)(\d\d\d\d\s\d\d\s\d\d)|()"   

def re_extract(pattern, txt): #regex
    try:
        matches = re.findall(pattern, txt)
        tmp_matches = []
        for match in matches:
            for tup in match:
                year, first, second = tup.split(' ')
                num=float(first)+float(second)/(float(year[2:4]))

                return str(num)
                if tup != '':
                    tmp_matches.append(tup)
        return list(set(tmp_matches))
    except:
        return txt



# Just merging everything into one function
def final_done(df,col):
    return df[col].apply(lambda measure: re.sub(r'in\.|[a-z]|[A-z]|\"|�|•|∩┐╜|\+|°|,|lb.|in\.|Ã‚Â·|\(|\)|#','',str(measure))).apply(lambda measure: re.sub(r'00:00:00|00:00|0:00','',str(measure))).apply(lambda measure: measure.rstrip()).apply(lambda x: remove_basic_date(x)).apply(lambda x: remove_date_year_first(x)).apply(lambda x: remove_date_year_last(x)).apply(lambda measure: re.sub(r'(?<=\d)\s?-',' ',str(measure))).apply(lambda x: date_to_decimal(x)).apply(lambda x: remove_fraction(x)).apply(lambda x: remove_frac(x)).apply(lambda x: re_extract(regex_year_month_day,x)).apply(lambda x: round_string_float(x))
#     return df[col].apply(lambda measure: re.sub(r'in\.|[a-z]|[A-z]|\"|�|•|∩┐╜|\+|°|,|lb.|in\.|Ã‚Â·|\(|\)','',str(measure))).apply(lambda measure: re.sub(r'00:00:00|00:00|0:00','',str(measure))).apply(lambda measure: measure.rstrip()).apply(lambda measure: re.sub(r'\-',' ',str(measure))).apply(lambda x: date_to_decimal(x)).apply(lambda x: round_string_float(x))
    
def final_done(df,col):
    return df[col].apply(lambda measure: re.sub(r'in\.|[a-z]|[A-z]|\"|�|•|∩┐╜|\+|°|,|lb.|in\.|Ã‚Â·|\(|\)|#','',str(measure))).apply(lambda measure: re.sub(r'00:00:00|00:00|0:00','',str(measure))).apply(lambda measure: measure.rstrip()).apply(lambda x: remove_basic_date(x)).apply(lambda x: remove_date_year_first(x)).apply(lambda x: remove_date_year_last(x)).apply(lambda measure: re.sub(r'(?<=\d)\s?-',' ',str(measure))).apply(lambda x: date_to_decimal(x)).apply(lambda x: remove_fraction(x)).apply(lambda x: remove_frac(x)).apply(lambda x: re_extract(regex_year_month_day,x)).apply(lambda x: round_string_float(x))
#     return df[col].apply(lambda measure: re.sub(r'in\.|[a-z]|[A-z]|\"|�|•|∩┐╜|\+|°|,|lb.|in\.|Ã‚Â·|\(|\)','',str(measure))).apply(lambda measure: re.sub(r'00:00:00|00:00|0:00','',str(measure))).apply(lambda measure: measure.rstrip()).apply(lambda measure: re.sub(r'\-',' ',str(measure))).apply(lambda x: date_to_decimal(x)).apply(lambda x: round_string_float(x))
    
def multi_conversion(numbers):  
#     try:
#         if (numbers.find('"') > -1 or numbers.find('in') > -1) and numbers.find('ft') > -1:
#             try:
#                 numbers=re.sub(r'ft\.','ft',str(numbers))
#                 numbers=re.sub(r'in\.','in',str(numbers))
#                 ft,inch=numbers.split('ft')
#                 ft=re.sub(r'yd\.?|in\.?|ft\.?|[a-z]|[A-z]|\"|�|•|∩┐╜|\+|°|','',str(ft))
#                 inch=re.sub(r'yd\.?|in\.?|ft\.?|[a-z]|[A-z]|\"|�|•|∩┐╜|\+|°|','',str(inch))
#                 ft_num=float(ft)*12
#                 total=ft_num+float(inch)
#                 return str(round_string_float(total))+' in'
#             except:
#                 return str(numbers)
#     except:
#         return numbers
                       
    if numbers.find('yd') > -1:
        try:
            yd=re.sub(r'yd\.?|in\.?|ft\.?|[a-z]|[A-z]|\"|�|•|∩┐╜|\+|°|','',str(numbers))
            new_yd=individual_value(yd)
#             new_yd=float(new_yd)*12*3
            return str(round_string_float(new_yd))+' yd'
        except:
            return numbers
    elif numbers.find('"') >-1:
        try:
            inch=re.sub(r'yd\.?|in\.?|ft\.?|[a-z]|[A-z]|\"|�|•|∩┐╜|\+|°|','',str(numbers))
            new_inch=individual_value(inch)
            return str(round_string_float(new_inch))+' in'
        except:
            return numbers
    elif numbers.find('in') >-1:
        try:
            inch=re.sub(r'yd\.?|in\.?|ft\.?|[a-z]|[A-z]|\"|�|•|∩┐╜|\+|°|','',str(numbers))
            new_inch=individual_value(inch)
            return str(round_string_float(new_inch))+' in'
        except:
            return numbers
        
    elif numbers.find('ft') > -1 or numbers.find("'") > -1 :
        try:
            ft=re.sub(r"yd\.?|in\.?|ft\.?|[a-z]|[A-z]|\'|�|•|∩┐╜|\+|°|",'',str(numbers))
            new_ft=individual_value(ft)
#             new_ft=float(new_ft)*12
            return str(round_string_float(new_ft))+' ft'
        except:
            return numbers
        
    elif numbers.find('mm') >-1:
        try:
            inch=re.sub(r'yd\.?|in\.?|ft\.?|[a-z]|[A-z]|\"|�|•| |∩┐╜|\+|°|','',str(numbers))
            new_inch=individual_value(inch)
            new_inch=float(new_inch)
            return str(round_string_float(new_inch))+' mm'
        except:
            return numbers
        
    elif numbers.find('m') >-1 or numbers.find('M') >-1:
        try:
            inch=re.sub(r'yd\.?|in\.?|ft\.?|[a-z]|[A-z]|\"|�|•| |∩┐╜|\+|°|','',str(numbers))
            new_inch=individual_value(inch)
            new_inch=float(new_inch)
            return str(round_string_float(new_inch))+' m'
        except:
            return numbers
    
    elif numbers.find('gal') >-1:
        try:
            inch=re.sub(r'gal\.|oz\.|yd\.?|in\.?|ft\.?|[a-z]|[A-z]|\"|�|•| |∩┐╜|\+|°|','',str(numbers))
            new_inch=individual_value(inch)
            new_inch=float(new_inch)
            return str(round_string_float(new_inch))+' gal'
        except:
            return numbers
    elif numbers.find('oz') >-1:
        try:
            inch=re.sub(r'min\.|hr\.|gal\.|oz\.|yd\.?|in\.?|ft\.?|[a-z]|[A-z]|\"|�|•| |∩┐╜|\+|°|','',str(numbers))
            new_inch=individual_value(inch)
            new_inch=float(new_inch)
            return str(round_string_float(new_inch))+' oz'
        except:
            return numbers
    else:
        return numbers
    
def individual_value(one):
    new_one=round_string_float(re_extract(regex_year_month_day,(remove_frac(remove_fraction(date_to_decimal(re.sub(r'(?<=\d)\s?-',' ',str(remove_date_year_last(remove_date_year_first(remove_basic_date(re.sub(r'00:00:00|00:00|0:00','',str(re.sub(r'in\.|[a-z]|[A-z]|\"|�|•|∩┐╜|\+|°|,|lb.|in\.|Ã‚Â·|\(|\)','',str(one)))).rstrip())))))))))))
    return new_one

In [3]:
customer_id = '5'
customer_name='%bedbathandbeyond%'
buckets = "Fixings & Fasteners"
strategy_version_input=702
attribute_id_input=2425
formatted_attribute = 'length'

In [4]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [5]:
buckets = [x for x in enrich_db(f'''SELECT * FROM (SELECT * FROM strategy_buckets sb WHERE strategy_version_id = {strategy_version_input}) AS active_strat
INNER JOIN strategy_buckets_attributes sba ON active_strat.id = sba.strategy_bucket_id
WHERE sba.attribute_id = {attribute_id_input}
AND sba.status = 'OPT_IN' ''')['bucket_id'].unique()]
# buckets

In [6]:
regex_pattern = r'(?i)((?<!(cord))\smeasures\s?\d+.?\"\s?L)|((?<!(cord))\smeasures\s\d+\.\d*.?\"\s?L)|()'
regex_pattern_new = r'(i?)(\d*\\\"\s?H\b)|(\d*\.\d*\\\"\s?H\b)|()'
regex_pattern_len_junk = r'(i?)(\d*\\\"\s?L\b)|(\d*\.\d*\\\"\s?L\b)|()'

In [7]:
df = query_from_file(file_name='./query/uncurated.sql', params=params)
print(len(df)) 
df = df[df['bucket_id'].isin(buckets)]
len(df)

89552


18252

In [8]:
# df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
# re_match=df[df['matches'].astype(str)!='[]']

In [18]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_len_junk, x))
matches=df[df['matches'].astype(str)!='[]']

In [19]:
multi_conversion('27.5" H')

'27.5 in'

In [ ]:
matches['final_matches']=matches['matches'].apply(lambda x: multi_conversion(str(x)))
matches

In [15]:
x['new_matches']= x['matches'].apply(
    lambda x: sorted(x, key=natural_key))
x.head(3)

product_id external_id  \
16   988529500    62723350   
17   817475429    62159210   
29  1157013454    61762688   

                                                                                        product_name  \
16  Playroom Entertainment Killer Bunnies and the Quest for the Magic Carrot: Booster Deck in Orange   
17                                               Steve Silver Co. Aerin Metal Serving Cart in Chrome   
29                                                                   Vornado® Personal Vortex Heater   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               long_desc  \
16                                                                                                                                                                                                                               Bring some extra wild fun to your Killer Bunnies and the Quest for the Magic Carrot game with this Booster Pack by Playroom Entertainment. Adds a hefty 55 cards to your existing set, and 6 colored pawns. Product Disclaimer/Warning: WARNING: CHOKING HAZARD - Small parts. Not for children under 3 years of age. Suitable for ages 12 years and up Includes: 55 Cards, 6 Colored Pawns Game Instructions Measures 6\" L x 5\" W x 1.5\" H Imported   
17  Durable, functional, and beautifully designed, the Aerin Serving Cart from Steve Silver Co. adds modern styling to any space in your home. This impressive piece features tempered glass shelving and bold chrome frame, with ample storage and display space. Beautiful serving carts perfect for home entertainment 2 shelves for serving and display Casters included for easy mobility Stemware storage under top shelf Three-ring wine bottle holder on bottom shelf Indoor use only Bold chrome finished frame Acrylic/glass/metal Wipe clean Measures 29\" L x 17\" W x 35\" H Weighs 34 lb. Maximum weight capacity: 150 lb. 1-year limited manufacturer's warranty Imported   
29                                                                                                    Perfect for use under a desk or on a tabletop, the Vornado Personal Vortex Heater circulates heat around your personal space. With two heat settings and a fan setting, it provides personalized comfort. The Vornado Personal Vortex Heater uses Vortex technology to circulate heat around your personal space Perfect for use under a desk or on a tabletop Two heat settings and fan-only mode for year-round comfort Tip-over protection Automatic safety shut-off system Measures 7.85\" L x 7.82\" W x 7.05\" H 3-year manufacturer's limited warranty Imported Model VH203   

    customer_id attributes                   buckets  bucket_id values  \
16            5       None  Card Games & Accessories       8529   None   
17            5       None     Serving/Rolling Carts       7489   None   
29            5       None         Heating Equipment       7355   None   

                 matches           new_matches  
16     [ Measures 6\" L]     [ Measures 6\" L]  
17    [ Measures 29\" L]    [ Measures 29\" L]  
29  [ Measures 7.85\" L]  [ Measures 7.85\" L]

In [16]:
x['new_matches']= x['new_matches'].apply(
    lambda x: sorted(x, key=natural_key))
x.head(3)

product_id external_id  \
16   988529500    62723350   
17   817475429    62159210   
29  1157013454    61762688   

                                                                                        product_name  \
16  Playroom Entertainment Killer Bunnies and the Quest for the Magic Carrot: Booster Deck in Orange   
17                                               Steve Silver Co. Aerin Metal Serving Cart in Chrome   
29                                                                   Vornado® Personal Vortex Heater   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               long_desc  \
16                                                                                                                                                                                                                               Bring some extra wild fun to your Killer Bunnies and the Quest for the Magic Carrot game with this Booster Pack by Playroom Entertainment. Adds a hefty 55 cards to your existing set, and 6 colored pawns. Product Disclaimer/Warning: WARNING: CHOKING HAZARD - Small parts. Not for children under 3 years of age. Suitable for ages 12 years and up Includes: 55 Cards, 6 Colored Pawns Game Instructions Measures 6\" L x 5\" W x 1.5\" H Imported   
17  Durable, functional, and beautifully designed, the Aerin Serving Cart from Steve Silver Co. adds modern styling to any space in your home. This impressive piece features tempered glass shelving and bold chrome frame, with ample storage and display space. Beautiful serving carts perfect for home entertainment 2 shelves for serving and display Casters included for easy mobility Stemware storage under top shelf Three-ring wine bottle holder on bottom shelf Indoor use only Bold chrome finished frame Acrylic/glass/metal Wipe clean Measures 29\" L x 17\" W x 35\" H Weighs 34 lb. Maximum weight capacity: 150 lb. 1-year limited manufacturer's warranty Imported   
29                                                                                                    Perfect for use under a desk or on a tabletop, the Vornado Personal Vortex Heater circulates heat around your personal space. With two heat settings and a fan setting, it provides personalized comfort. The Vornado Personal Vortex Heater uses Vortex technology to circulate heat around your personal space Perfect for use under a desk or on a tabletop Two heat settings and fan-only mode for year-round comfort Tip-over protection Automatic safety shut-off system Measures 7.85\" L x 7.82\" W x 7.05\" H 3-year manufacturer's limited warranty Imported Model VH203   

    customer_id attributes                   buckets  bucket_id values  \
16            5       None  Card Games & Accessories       8529   None   
17            5       None     Serving/Rolling Carts       7489   None   
29            5       None         Heating Equipment       7355   None   

                 matches           new_matches  
16     [ Measures 6\" L]     [ Measures 6\" L]  
17    [ Measures 29\" L]    [ Measures 29\" L]  
29  [ Measures 7.85\" L]  [ Measures 7.85\" L]

In [19]:
replacement = {
    'a': '',
    'b': '',
    'c': '',
    'd': '',
    'e': '',
    'f': '',
    'g': '',
    'h': '',
    'i': '',
    'j': '',
    'k': '',
    'l': '',
    'm': '',
    'n': '',
    'o': '',
    'p': '',
    'q': '',
    
    'r': '',
    's': '',
    't': '',
    'u': '',
    'v': '',
    'w': '',
    'x': '',
    'y': '',
    'z': '',
    'A': '',
    'k': '',
    'B': '',
    'C': '',
    'D': '',
    'E': '',
    'F': '',
    'G': '',
    'H': '',
    'I': '',
    'J': '',
    'K': '',
    'L': '',
    'M': '',
    'N': '',
    'O': '',
    'P': '',
    'Q': '',
    'R': '',
    'S': '',
    'T': '',
    'U': '',
    'V': '',
    'W': '',
    'X': '',
    'Y': '',
    'Z': '',
    '-  ': '',
    ' ': '',
    '  ': '',
    " ": "", 
    "-": "",
    "  ": "",
    '4225': "42.25",

    
    
    
    
    
    
    
#     '\\\"': " in"
#     '\" L': " in",
#     '\" L': " in",
#     '\" L ': " in",
#     "\ L": " in",
#     '\" L': " in",
#     '\" L': " in",
#     '\" L': " in",
#     '\"L': " in",    
}

In [20]:
# Replacement
x['new_matches']= x['new_matches'].apply(
    lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)

In [21]:
# Replacement
x['new_matches']= x['new_matches'].apply(
    lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)
x.head(5)

product_id external_id  \
16   988529500    62723350   
17   817475429    62159210   
29  1157013454    61762688   
33   817474204    61732766   
34   817474203    61732759   

                                                                                        product_name  \
16  Playroom Entertainment Killer Bunnies and the Quest for the Magic Carrot: Booster Deck in Orange   
17                                               Steve Silver Co. Aerin Metal Serving Cart in Chrome   
29                                                                   Vornado® Personal Vortex Heater   
33                                                  NFL New England Patriots 3-Way Diztracto Spinner   
34                                                         NFL New York Jets 3-Way Diztracto Spinner   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 long_desc  \
16                                                                                                                                                                                                                                                                                 Bring some extra wild fun to your Killer Bunnies and the Quest for the Magic Carrot game with this Booster Pack by Playroom Entertainment. Adds a hefty 55 cards to your existing set, and 6 colored pawns. Product Disclaimer/Warning: WARNING: CHOKING HAZARD - Small parts. Not for children under 3 years of age. Suitable for ages 12 years and up Includes: 55 Cards, 6 Colored Pawns Game Instructions Measures 6\" L x 5\" W x 1.5\" H Imported   
17                                                    Durable, functional, and beautifully designed, the Aerin Serving Cart from Steve Silver Co. adds modern styling to any space in your home. This impressive piece features tempered glass shelving and bold chrome frame, with ample storage and display space. Beautiful serving carts perfect for home entertainment 2 shelves for serving and display Casters included for easy mobility Stemware storage under top shelf Three-ring wine bottle holder on bottom shelf Indoor use only Bold chrome finished frame Acrylic/glass/metal Wipe clean Measures 29\" L x 17\" W x 35\" H Weighs 34 lb. Maximum weight capacity: 150 lb. 1-year limited manufacturer's warranty Imported   
29                                                                                                                                                      Perfect for use under a desk or on a tabletop, the Vornado Personal Vortex Heater circulates heat around your personal space. With two heat settings and a fan setting, it provides personalized comfort. The Vornado Personal Vortex Heater uses Vortex technology to circulate heat around your personal space Perfect for use under a desk or on a tabletop Two heat settings and fan-only mode for year-round comfort Tip-over protection Automatic safety shut-off system Measures 7.85\" L x 7.82\" W x 7.05\" H 3-year manufacturer's limited warranty Imported Model VH203   
33  One of the trendiest gadgets, the NFL 3-Way Diztracto Spinner is a durable pocket-size toy that helps enhance focus and relieve stress while also displaying your team spirit. It features your favorite team logo and colors and is perfect for home or work. Keep yourself entertained while displaying your pro football team p

In [22]:
replacement = {
    '\\\"': ' in',
    '\, ' :','
}

In [23]:
# # Replacement
# x['new_matches']= x['new_matches'].apply(
#     lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)

In [24]:
# Replacement
x['new_matches']= x['new_matches'].apply(
    lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)
x.head(5)

product_id external_id  \
16   988529500    62723350   
17   817475429    62159210   
29  1157013454    61762688   
33   817474204    61732766   
34   817474203    61732759   

                                                                                        product_name  \
16  Playroom Entertainment Killer Bunnies and the Quest for the Magic Carrot: Booster Deck in Orange   
17                                               Steve Silver Co. Aerin Metal Serving Cart in Chrome   
29                                                                   Vornado® Personal Vortex Heater   
33                                                  NFL New England Patriots 3-Way Diztracto Spinner   
34                                                         NFL New York Jets 3-Way Diztracto Spinner   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 long_desc  \
16                                                                                                                                                                                                                                                                                 Bring some extra wild fun to your Killer Bunnies and the Quest for the Magic Carrot game with this Booster Pack by Playroom Entertainment. Adds a hefty 55 cards to your existing set, and 6 colored pawns. Product Disclaimer/Warning: WARNING: CHOKING HAZARD - Small parts. Not for children under 3 years of age. Suitable for ages 12 years and up Includes: 55 Cards, 6 Colored Pawns Game Instructions Measures 6\" L x 5\" W x 1.5\" H Imported   
17                                                    Durable, functional, and beautifully designed, the Aerin Serving Cart from Steve Silver Co. adds modern styling to any space in your home. This impressive piece features tempered glass shelving and bold chrome frame, with ample storage and display space. Beautiful serving carts perfect for home entertainment 2 shelves for serving and display Casters included for easy mobility Stemware storage under top shelf Three-ring wine bottle holder on bottom shelf Indoor use only Bold chrome finished frame Acrylic/glass/metal Wipe clean Measures 29\" L x 17\" W x 35\" H Weighs 34 lb. Maximum weight capacity: 150 lb. 1-year limited manufacturer's warranty Imported   
29                                                                                                                                                      Perfect for use under a desk or on a tabletop, the Vornado Personal Vortex Heater circulates heat around your personal space. With two heat settings and a fan setting, it provides personalized comfort. The Vornado Personal Vortex Heater uses Vortex technology to circulate heat around your personal space Perfect for use under a desk or on a tabletop Two heat settings and fan-only mode for year-round comfort Tip-over protection Automatic safety shut-off system Measures 7.85\" L x 7.82\" W x 7.05\" H 3-year manufacturer's limited warranty Imported Model VH203   
33  One of the trendiest gadgets, the NFL 3-Way Diztracto Spinner is a durable pocket-size toy that helps enhance focus and relieve stress while also displaying your team spirit. It features your favorite team logo and colors and is perfect for home or work. Keep yourself entertained while displaying your pro football team p

In [25]:
replacement = {
    'in': ' in',
}

In [26]:
# Replacement
x['new_matches']= x['new_matches'].apply(
    lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)
x.head(5)

product_id external_id  \
16   988529500    62723350   
17   817475429    62159210   
29  1157013454    61762688   
33   817474204    61732766   
34   817474203    61732759   

                                                                                        product_name  \
16  Playroom Entertainment Killer Bunnies and the Quest for the Magic Carrot: Booster Deck in Orange   
17                                               Steve Silver Co. Aerin Metal Serving Cart in Chrome   
29                                                                   Vornado® Personal Vortex Heater   
33                                                  NFL New England Patriots 3-Way Diztracto Spinner   
34                                                         NFL New York Jets 3-Way Diztracto Spinner   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 long_desc  \
16                                                                                                                                                                                                                                                                                 Bring some extra wild fun to your Killer Bunnies and the Quest for the Magic Carrot game with this Booster Pack by Playroom Entertainment. Adds a hefty 55 cards to your existing set, and 6 colored pawns. Product Disclaimer/Warning: WARNING: CHOKING HAZARD - Small parts. Not for children under 3 years of age. Suitable for ages 12 years and up Includes: 55 Cards, 6 Colored Pawns Game Instructions Measures 6\" L x 5\" W x 1.5\" H Imported   
17                                                    Durable, functional, and beautifully designed, the Aerin Serving Cart from Steve Silver Co. adds modern styling to any space in your home. This impressive piece features tempered glass shelving and bold chrome frame, with ample storage and display space. Beautiful serving carts perfect for home entertainment 2 shelves for serving and display Casters included for easy mobility Stemware storage under top shelf Three-ring wine bottle holder on bottom shelf Indoor use only Bold chrome finished frame Acrylic/glass/metal Wipe clean Measures 29\" L x 17\" W x 35\" H Weighs 34 lb. Maximum weight capacity: 150 lb. 1-year limited manufacturer's warranty Imported   
29                                                                                                                                                      Perfect for use under a desk or on a tabletop, the Vornado Personal Vortex Heater circulates heat around your personal space. With two heat settings and a fan setting, it provides personalized comfort. The Vornado Personal Vortex Heater uses Vortex technology to circulate heat around your personal space Perfect for use under a desk or on a tabletop Two heat settings and fan-only mode for year-round comfort Tip-over protection Automatic safety shut-off system Measures 7.85\" L x 7.82\" W x 7.05\" H 3-year manufacturer's limited warranty Imported Model VH203   
33  One of the trendiest gadgets, the NFL 3-Way Diztracto Spinner is a durable pocket-size toy that helps enhance focus and relieve stress while also displaying your team spirit. It features your favorite team logo and colors and is perfect for home or work. Keep yourself entertained while displaying your pro football team p

In [27]:
matches=x
matches.head(3)

product_id external_id  \
16   988529500    62723350   
17   817475429    62159210   
29  1157013454    61762688   

                                                                                        product_name  \
16  Playroom Entertainment Killer Bunnies and the Quest for the Magic Carrot: Booster Deck in Orange   
17                                               Steve Silver Co. Aerin Metal Serving Cart in Chrome   
29                                                                   Vornado® Personal Vortex Heater   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               long_desc  \
16                                                                                                                                                                                                                               Bring some extra wild fun to your Killer Bunnies and the Quest for the Magic Carrot game with this Booster Pack by Playroom Entertainment. Adds a hefty 55 cards to your existing set, and 6 colored pawns. Product Disclaimer/Warning: WARNING: CHOKING HAZARD - Small parts. Not for children under 3 years of age. Suitable for ages 12 years and up Includes: 55 Cards, 6 Colored Pawns Game Instructions Measures 6\" L x 5\" W x 1.5\" H Imported   
17  Durable, functional, and beautifully designed, the Aerin Serving Cart from Steve Silver Co. adds modern styling to any space in your home. This impressive piece features tempered glass shelving and bold chrome frame, with ample storage and display space. Beautiful serving carts perfect for home entertainment 2 shelves for serving and display Casters included for easy mobility Stemware storage under top shelf Three-ring wine bottle holder on bottom shelf Indoor use only Bold chrome finished frame Acrylic/glass/metal Wipe clean Measures 29\" L x 17\" W x 35\" H Weighs 34 lb. Maximum weight capacity: 150 lb. 1-year limited manufacturer's warranty Imported   
29                                                                                                    Perfect for use under a desk or on a tabletop, the Vornado Personal Vortex Heater circulates heat around your personal space. With two heat settings and a fan setting, it provides personalized comfort. The Vornado Personal Vortex Heater uses Vortex technology to circulate heat around your personal space Perfect for use under a desk or on a tabletop Two heat settings and fan-only mode for year-round comfort Tip-over protection Automatic safety shut-off system Measures 7.85\" L x 7.82\" W x 7.05\" H 3-year manufacturer's limited warranty Imported Model VH203   

    customer_id attributes                   buckets  bucket_id values  \
16            5       None  Card Games & Accessories       8529   None   
17            5       None     Serving/Rolling Carts       7489   None   
29            5       None         Heating Equipment       7355   None   

                 matches new_matches  
16     [ Measures 6\" L]     [6  in]  
17    [ Measures 29\" L]    [29  in]  
29  [ Measures 7.85\" L]  [7.85  in]

In [28]:
#import regex as re
matches[curation_col]=matches['new_matches'].astype(str)
matches[curation_col]=matches[curation_col].apply(lambda x: f'"{x}"')
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\" ','"',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\, ',',',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\\",'',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"  ",' ',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
matches.head(3)

product_id external_id  \
16   988529500    62723350   
17   817475429    62159210   
29  1157013454    61762688   

                                                                                        product_name  \
16  Playroom Entertainment Killer Bunnies and the Quest for the Magic Carrot: Booster Deck in Orange   
17                                               Steve Silver Co. Aerin Metal Serving Cart in Chrome   
29                                                                   Vornado® Personal Vortex Heater   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               long_desc  \
16                                                                                                                                                                                                                               Bring some extra wild fun to your Killer Bunnies and the Quest for the Magic Carrot game with this Booster Pack by Playroom Entertainment. Adds a hefty 55 cards to your existing set, and 6 colored pawns. Product Disclaimer/Warning: WARNING: CHOKING HAZARD - Small parts. Not for children under 3 years of age. Suitable for ages 12 years and up Includes: 55 Cards, 6 Colored Pawns Game Instructions Measures 6\" L x 5\" W x 1.5\" H Imported   
17  Durable, functional, and beautifully designed, the Aerin Serving Cart from Steve Silver Co. adds modern styling to any space in your home. This impressive piece features tempered glass shelving and bold chrome frame, with ample storage and display space. Beautiful serving carts perfect for home entertainment 2 shelves for serving and display Casters included for easy mobility Stemware storage under top shelf Three-ring wine bottle holder on bottom shelf Indoor use only Bold chrome finished frame Acrylic/glass/metal Wipe clean Measures 29\" L x 17\" W x 35\" H Weighs 34 lb. Maximum weight capacity: 150 lb. 1-year limited manufacturer's warranty Imported   
29                                                                                                    Perfect for use under a desk or on a tabletop, the Vornado Personal Vortex Heater circulates heat around your personal space. With two heat settings and a fan setting, it provides personalized comfort. The Vornado Personal Vortex Heater uses Vortex technology to circulate heat around your personal space Perfect for use under a desk or on a tabletop Two heat settings and fan-only mode for year-round comfort Tip-over protection Automatic safety shut-off system Measures 7.85\" L x 7.82\" W x 7.05\" H 3-year manufacturer's limited warranty Imported Model VH203   

    customer_id attributes                   buckets  bucket_id values  \
16            5       None  Card Games & Accessories       8529   None   
17            5       None     Serving/Rolling Carts       7489   None   
29            5       None         Heating Equipment       7355   None   

                 matches new_matches     Q:length  
16     [ Measures 6\" L]     [6  in]     ["6 in"]  
17    [ Measures 29\" L]    [29  in]    ["29 in"]  
29  [ Measures 7.85\" L]  [7.85  in]  ["7.85 in"]

In [30]:
y.head(3)

product_id external_id  \
4   1345561951    63245929   
10   988795875    62245265   
16   988529500    62723350   

                                                                                        product_name  \
4               Classic Accessories® Madrona RainProof Patio Chaise Lounge Chair Cover in Dark Cocoa   
10                                                         Steve Silver Co. Ambrose Square End Table   
16  Playroom Entertainment Killer Bunnies and the Quest for the Magic Carrot: Booster Deck in Orange   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  long_desc  \
4   The Classic Accessories Madrona RainProof Patio Chaise Lounge Chair Cover is perfect for protecting your patio chair from the elements and keeping it ready for guests. The cover features cord-lock closures and designer webbing for an easy and snug fit. Perfect for protecting your patio chair against the elements and ensuring that it is ready for guests RainProof™ fabric features taped seam construction to keep rain out Customized click-close buckles and adjustable designer webbing provide exceptional security in windy conditions Vent hoods are seam-taped to provide maximum ventilation and minimum condensation Cord-lock closures allow for a snug fit Custom designer handles allow for hassle-free installation and removal Not designed to entirely cover the wheels or legs 100% polyester Hand wash Cover fits chaise lounge chairs measuring 66\" L x 28\" W x 27.5\" H 3-year limited manufacturer's warranty Imported   
10                                                                                                                                                                                                                                                                                                                                                                                 Rustic industrial design makes the Ambrose End Table from Steve Silver Co. a unique and stylish accent piece in your home. Features a butcher block wood tabletop supported by a dark iron frame with rivet details and demilune stretchers. Bring unique industrial style to your décor Butcher block style wood tabletop Industrial dark iron legs Crescent-shaped demilune stretchers Rivet details Wood/iron/wood veneer Wipe clean Easy to assemble Measures 22\" D x 22\" W x 24\" H Weighs 24 lb. 150 lb. weight capacity 1-year manufacturer's warranty Imported   
16                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  Bring some extra wild fun to your Killer Bunnies and the Quest for the Magic Carrot game with this Booster Pack by Playroom Entertainment. Adds a hefty 55 cards to your existing set, and 6 colored pawn

In [31]:
matchez=y
matchez[curation_col]=matchez['new_matches'].astype(str)
matchez[curation_col]=matchez[curation_col].apply(lambda x: f'"{x}"')
matchez[curation_col]=matchez[curation_col].apply(lambda measure: re.sub(r'\" H',' in',str(measure)))
matchez[curation_col]=matchez[curation_col].apply(lambda measure: re.sub(r'\\','',str(measure)))
matchez[curation_col]=matchez[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
# matchez[curation_col]=matchez[curation_col].apply(lambda measure: re.sub(r"\\",'',str(measure)))
matchez[curation_col]=matchez[curation_col].apply(lambda measure: re.sub(r" ",'',str(measure)))
matchez

product_id external_id  \
4      1345561951    63245929   
10      988795875    62245265   
16      988529500    62723350   
17      817475429    62159210   
29     1157013454    61762688   
...           ...         ...   
89438   988546708    66538233   
89444   817491271    62064613   
89446   817421424    69536981   
89447   817421550    69536982   
89448   988551111    67657445   

                                                                                           product_name  \
4                  Classic Accessories® Madrona RainProof Patio Chaise Lounge Chair Cover in Dark Cocoa   
10                                                            Steve Silver Co. Ambrose Square End Table   
16     Playroom Entertainment Killer Bunnies and the Quest for the Magic Carrot: Booster Deck in Orange   
17                                                  Steve Silver Co. Aerin Metal Serving Cart in Chrome   
29                                                                      Vornado® Personal Vortex Heater   
...                                                                                                 ...   
89438                                                             Connex Tedco Kids® Piano Learning Kit   
89444                                             Red Rock Outdoor Gear Diplomat Backpack in Olive Drab   
89446                                        My First Easter Personalized Bunny in Grey with Blue Shirt   
89447                                        My First Easter Personalized Bunny in Grey with Pink Shirt   
89448                                                      BePuzzled Flamingo 210-Piece 3D Pixel Puzzle   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  long_desc  \
4                                                                                                                                                                                                                                                                   The Classic Accessories Madrona RainProof Patio Chaise Lounge Chair Cover is perfect for protecting your patio chair from the elements and keeping it ready for guests. The cover features cord-lock closures and designer webbing for an easy and snug fit. Perfect for protecting your patio chair against the elements and ensuring that it is ready for guests RainProof™ fabric features taped seam construction to keep rain out Customized click-close buckles and adjustable designer webbing provide exceptional security in windy conditions Vent hoods are seam-taped to provide maximum ventilation and minimum condensation Cord-lock closures allow for a snug fit Custom designer handles allow for hassle-free installation and removal Not designed to entirely cover the wheels or legs 100% polyester Hand wash Cover fits chaise lounge chairs measuring 66\" L x 28\" W x 27.5\" H 3-year lim

In [32]:
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"xa0",'',str(measure)))

In [33]:
len(matches)

9477

In [37]:
y=y[y['new_matches_junk'].astype(str)=='[]']
y[y['buckets'].astype(str)=='Prints, Posters, & Paintings'].head(3) # \d*H works

product_id external_id  \
52    1346701433    61395060   
2430   817589851    67215553   
2431   817350356    67215416   

                                                                     product_name  \
52                         Poppies Floral Canvas Wall Art in Red/White (Set of 2)   
2430  Courtside Market™ Winter Snowman I 16-Inch x 1.5-Inch Framed Wrapped Canvas   
2431   Courtside Market™ Merry Christmas 16-Inch x 1.5-Inch Framed Wrapped Canvas   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   long_desc  \
52    The Floral Canvas Wall Art Collection showcases elegant blooms which lend rich textures and soft, natural hues to your décor. Hand embellished detailing injects painterly appeal into any room in your home. Wall mounted and easy to hang. The Poppies Floral Canvas Wall Art showcases elegant blooms which lend rich textures and soft, natural hues to your décor Set of 2 Hand embellished for authenticity MDF/canvas/gel/ink/wood Measures 12\" W x 12\" H x 1.26\" D Imported   
2430                                                                                                                             Classic artistry joins this durable and easy-care piece in the Novelty by Courtside Market.The wrapped canvas is designed with impressive colors that will match beautifully to indoor elements and any decor. Finished with a beige frame Canvas Shape: Other Portrait/Landscape Orientation Measures 16\"W x 1.5\"H Made in the USA of imported materials   
2431                                                                                                                                                                         The design of this durable wrapped canvas adds a touch of drama and elegance to any room in your home. Colorful and stylish, this piece will go great in any room. Finished with a beige frame Canvas Shape: Other Portrait/Landscape Orientation Measures 16\"W x 1.5\"H Made in the USA of imported materials   

      customer_id attributes                       buckets  bucket_id values  \
52              5       None  Prints, Posters, & Paintings       8130   None   
2430            5       None  Prints, Posters, & Paintings       8130   None   
2431            5       None  Prints, Posters, & Paintings       8130   None   

     matches new_matches_junk new_matches     Q:length  
52        []               []    [12\" H]   "["12in"]"  
2430      []               []    [1.5\"H]  "["1.5"H"]"  
2431      []               []    [1.5\"H]  "["1.5"H"]"

In [38]:
y[y['buckets'].astype(str)=='Side/End Tables'][0:500] #Diameter or D

product_id external_id  \
10      988795875    62245265   
78     1348005739    47339835   
3235   1345420009    65273586   
3718   1345405872    63930337   
6048   1345369304    45954085   
6049   1345369303    45954061   
7271   1332554812    69745457   
9032    817673383    66067849   
11315  1338761808    69642198   
11506  1347990760    17626221   
14441  1345555829    60715234   
14612  1336354224    63343113   
17221  1341290532    64470528   
18572  1338798910    69761203   
19342   817486727    46476036   
22502  1347996300    40666341   
22519  1345543725    40160399   
22550  1345539346    18395908   
22563  1345537629    16144932   
23047  1330671730    69727544   
24605  1345405871    63930320   
26619   817640787    69546547   
28190   817269606    47690806   
31724  1347806909    40783185   
31725  1332782858    69707352   
31809  1338798952    69761159   
32625  1345544443    41195048   
36316   817673367    66067795   
37227  1332554939    69745471   
39627   988626671    63747003   
40299  1330441043    69751393   
42289  1338798957    69761210   
43823  1345555902    60775733   
44648  1348000949    44735043   
44694  1345544752    41518830   
44753  1347988653    14298100   
45213   817413332    69505776   
45279  1347977486    69727550   
46074  1345446827    67197316   
48254  1345369305    45954078   
51918  1330441044    69751394   
56287   988369875    64379845   
56362   988758804    69587992   
57440  1341415557    69761198   
57442  1340108383    69745477   
59824   817532534    44716554   
60274  1345404172    63746464   
63177  1330441042    69751390   
64951   988758817    69587888   
72957  1345544446    41195246   
74029   817351438    67197309   
74861   817675938    67190171   
76585   988763044    69612889   
77615  1329767388    69632450   
80867  1338798951    69761200   
80868  1338798889    69761182   
81763   817673382    66067832   
82220   817256351    61329300   
83671   817582116    66650430   
85649   817370418    68194093   
86991   817675885    67187690   
88608  1345559947    62590372   
88981   817673381    66067856   

                                                                     product_name  \
10                                      Steve Silver Co. Ambrose Square End Table   
78                                            Outdoor Mosaic Bistro Table in Blue   
3235                                         Weybridge Storage Side Table in Grey   
3718                                           Elephant Round Side Table in White   
6048                                             2-Tier Round Side Table in Black   
6049                                          2-Tier Round Side Table in Bungalow   
7271                         Simpli Home™ Corbin Metal Side Table in Antique Gold   
9032                                    Brassex 3-Piece Nesting Table Set in Grey   
11315                                     Baxton Studio Margaux End Table in Grey   
11506                                                         Umbrella Side Table   
14441                                       Two Tier Medallion Round Table in Red   
14612                          Butler Specialty Company Halcyon End Table in Teal   
17221                                 Safavieh Lorna 3-Drawer Nightstand in White   
18572                             Kate and Laurel® Razza Round Side Table in Gold   
19342                                          Adesso® Gibson End Table in Copper   
22502                                        Wicker Umbrella Side Table in Bronze   
22519                                            Ellington Side Tables (Set of 2)   
22550                               Palmetto Faux Marble Accent Tables - Set of 2   
22563                                                Harrison Walnut Accent Table   
23047              Ridge Road Décor Modern Metal and Rattan Accent Table in Black   
24605                                                Geo Round Side Table in Gold   
26619           A&B Home

In [39]:
y[y['buckets'].astype(str)=='Ottomans/Poufs/Cubes/Footrests'].head(3)

product_id external_id                              product_name  \
5705    817246884    60472717  Safavieh Arturo Storage Ottoman in Beige   
5842    817242802    47469297       Safavieh Roitfeld Ottoman in Orange   
11254  1347814744    44450939     Amelia Tufted Ottoman in Velvet Cocoa   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          long_desc  \
5705                                                                                                                                                                              The fashionable Safavieh Arturo Storage Ottoman is the perfect addition to any space. Featuring luxurious velvet upholstery, this exceptional ottoman boasts plush foam padding for added support and comfort and spacious interior storage. The Safavieh Arturo Storage Ottoman offers elegant organization in your home Bun feet, button tufting, and nail head border offers fascinating details Ample interior storage space Plush padding for added comfort and support Cotton velvet upholstery Birchwood construction Spot clean only No assembly required Measures 18.9\" W x 18.9\" D x 20.1\" H Weighs 25.3 lb. 135 lb. weight capacity 1-year limited manufacturer's warranty Imported   
5842                                                                                                                                                                                                                                                                                                                                  Add a sophisticated contemporary look to your room with the Safavieh Roitfeld Ottoman. Featuring elegant faux leather upholstery with a button-tufted top and a polished stainless steel frame, blending high fashion and comfort in one luxurious design. Inspired by sleek geometric style from Barcelona, this Roitfeld Ottoman from Safavieh adds a polished contemporary look to any room Faux leather button-tufted top Polished stainless steel/PU Spot clean only Measures 25.2\" W x 18\" D x 18.9\" H No assembly required Imported   
11254  Featuring a generously padded top with handcrafted diamond tufting, the Amelia Tufted Ottoman injects comfort and style to any space. Elegant velvet polyester fabric upholstery plus elegantly turned, contrasting wood legs complete its charming design. Amelia Tufted Ottoman makes a stylish functional accent in any room courtesy of its attractive and plush design Handcrafted diamond tuft detailing Soft and elegant velvet polyester upholstery Generously cushioned top Sturdy and elegant turned legs with a contrasting dark brown finish Legs come with non-slip feet to help keep ottoman in place and prevent scratches on the floor Durable Rubberwood frame Wood and polyester fabric with foam fill construction Spot clean Easy to assemble Measures 16\" D x 20\" W x 17\" H Maximum weight capacity: 250 lb. 1-year manufacturer's warranty Imported   

       customer_id attributes                         buckets  bucket_id  \
5705             5       None  Ottomans/Poufs/Cubes/Footrests       7438   
5842             5       None  Ottomans/Poufs/Cubes/Footrests       7438   
11254            5       None  Ottomans/Poufs/Cubes/Footr

In [29]:
error

NameError: name 'error' is not defined

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',wipe)

# send to the folder for upload

In [ ]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/MultipleBuckets - {attribute}-- match upload_Shawn.csv',index=False)

In [ ]:
looks_good('Bed Bath & Beyond', attribute, df, matches)